In [130]:
from hotels50k import Hotels50kDataset, UseOriginalTestSplitManager, compress_serialize
from powerful_benchmarker.split_managers import IndexSplitManager, ClosedSetSplitManager
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
from tqdm import tqdm

In [131]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
dataset = Hotels50kDataset(root='../hotels50k', target='chains', download=False)

Loading image folders
Concatting dataset
Getting labels
Done loading dataset


In [133]:
len(dataset)

95968

In [134]:
len(dataset.original_train_dataset), len(dataset.original_test_dataset)

(85069, 10899)

In [135]:
dataset.labels

array([38, 38, 38, ..., 12, 12, 12])

In [136]:
dataset[858]

{'data': <PIL.Image.Image image mode=RGB size=640x512 at 0x7EFEA0B628B0>,
 'label': 79}

In [137]:
dataset.labels.shape

(95968,)

In [138]:
len(dataset.original_train_dataset), len(dataset.original_test_dataset)

(85069, 10899)

In [139]:
dataset.original_train_dataset.keys[-10:]

[b'85059',
 b'85060',
 b'85061',
 b'85062',
 b'85063',
 b'85064',
 b'85065',
 b'85066',
 b'85067',
 b'85068']

In [140]:
dataset.original_test_dataset.keys[:10]

[b'0', b'1', b'2', b'3', b'4', b'5', b'6', b'7', b'8', b'9']

In [141]:
import numpy as np

### Train len checks

In [142]:
len(dataset.train_targets)

85069

In [143]:
len(dataset.train_indices)

85069

In [144]:
len(dataset.original_train_dataset)

85069

In [145]:
assert len(dataset.original_train_dataset) == len(dataset.train_indices) == len(dataset.train_targets)

### Test len checks

In [146]:
len(dataset.test_targets), len(dataset.test_indices), len(dataset.original_test_dataset)

(10899, 10899, 10899)

In [147]:
assert len(dataset.test_targets) == len(dataset.test_indices) == len(dataset.original_test_dataset)

In [148]:
assert len(dataset) == len(dataset.dataset) == len(dataset.original_train_dataset)  + len(dataset.original_test_dataset) 

In [149]:
assert dataset[dataset.test_indices[1]]['data'] == dataset.original_test_dataset[1][0]

In [150]:
dataset.train_indices[-10:]

array([85059, 85060, 85061, 85062, 85063, 85064, 85065, 85066, 85067,
       85068])

In [151]:
dataset.test_indices[:10]

array([85069, 85070, 85071, 85072, 85073, 85074, 85075, 85076, 85077,
       85078])

In [152]:
dataset[85069]

{'data': <PIL.Image.Image image mode=RGB size=480x640 at 0x7EFEF1E8A820>,
 'label': 38}

In [153]:
dataset.original_train_dataset.keys[-10:]

[b'85059',
 b'85060',
 b'85061',
 b'85062',
 b'85063',
 b'85064',
 b'85065',
 b'85066',
 b'85067',
 b'85068']

In [154]:
dataset.original_test_dataset.keys[:10]

[b'0', b'1', b'2', b'3', b'4', b'5', b'6', b'7', b'8', b'9']

In [155]:
count = 0
for i in range(len(dataset.original_test_dataset)):
    try:
        #print(i, dataset.original_test_dataset[i])
        count += 1
    except Exception as e:
        pass
count

10899

In [156]:
len(dataset), len(dataset.original_train_dataset)

(95968, 85069)

In [158]:
count = 0
for i in dataset.test_indices:
    try:
#         print(i, dataset[i])
        count += 1
    except Exception as e:
        pass
#         print(e)
count

10899

In [159]:
len(dataset.test_chains), len(dataset.test_hotels)

(10899, 10899)

In [160]:
len(dataset.original_test_dataset)

10899

In [161]:
ds = dataset

# first level of dictionary is transform type
# second level consists of separate dataset objects for each split
datasets = {"eval": {"train": ds,
                    "val": ds,
                    "test": ds}
            }

helper_split_manager = UseOriginalTestSplitManager()
split_manager = ClosedSetSplitManager(num_training_partitions=4, num_training_sets=4, helper_split_manager=helper_split_manager)

split_manager.create_split_schemes(datasets)

test_set = split_manager.get_dataset("eval", "test")

for name in split_manager.split_scheme_names:
    split_manager.set_curr_split_scheme(name)
    print("CURR SPLIT SCHEME =", split_manager.curr_split_scheme_name)
    print("train len", len(split_manager.get_dataset("eval", "train")))
    print("val len", len(split_manager.get_dataset("eval", "val")))
    print("test len", len(split_manager.get_dataset("eval", "test")))

    print("train labels", split_manager.get_label_set("eval", "train"))
    print("val labels", split_manager.get_label_set("eval", "val"))
    print("test labels", split_manager.get_label_set("eval", "test"))
    
    assert set(split_manager.get_dataset("eval", "train").indices).intersection(split_manager.get_dataset("eval", "val").indices) == set()
    assert set(split_manager.get_dataset("eval", "train").indices).intersection(split_manager.get_dataset("eval", "test").indices) == set()
    assert set(split_manager.get_dataset("eval", "val").indices).intersection(split_manager.get_dataset("eval", "test").indices) == set()


CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_0
train len 63801
val len 21268
test len 10899
train labels {0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, -1}
val labels {0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, -1}
test labels {0, 1, 2, 3, 5, 6, 7, 10, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 4

In [162]:
train_ds = split_manager.get_dataset("eval", "train")
batch_size = 32

def collate(stuff):
    return [s['data'] for s in stuff]

loader = DataLoader(train_ds, shuffle=True, batch_size=batch_size,  num_workers=4, collate_fn=collate)

i = 5
for batch in tqdm(loader):
    assert len(batch) > 0
    i -= 1
    if i <= 0:
        break

  0%|          | 4/1994 [00:01<09:05,  3.65it/s]


In [163]:
test_ds = split_manager.get_dataset("eval", "test")
batch_size = 32

def collate(stuff):
    return [s['data'] for s in stuff]

loader = DataLoader(test_ds, shuffle=True, batch_size=batch_size,  num_workers=4, collate_fn=collate)

i = 5
for batch in tqdm(loader):
    assert len(batch) > 0
    i -= 1
    if i <= 0:
        break

  1%|          | 4/341 [00:01<01:35,  3.54it/s]
